# Feature engineering

So far we have been only using the close price and the relative differences as features.

In [1]:
cd ~/downloads/Pedro-Pineapple

/home/aargaez/downloads/Pedro-Pineapple


In [2]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv('experiments/SPY_data.csv')

In [ ]:
data